In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,2953
2,Huelva,Confirmados PDIA 14 días,1065
3,Huelva,Tasa PDIA 14 días,"208,51974476400068"
4,Huelva,Confirmados PDIA 7 días,673
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,1
629,Municipio de Huelva sin especificar,Total Confirmados,32
630,Municipio de Huelva sin especificar,Curados,8


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  2953.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  772.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 826 personas en los últimos 7 días 

Un positivo PCR cada 487 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,2953.0,673.0,1065.0,510743.0,131.768815,208.519745,197.0
Huelva-Costa,1513.0,329.0,559.0,288115.0,114.190514,194.019749,114.0
Huelva (capital),772.0,174.0,295.0,143663.0,121.116780,205.341668,72.0
Condado-Campiña,929.0,163.0,268.0,155057.0,105.122632,172.839665,54.0
Sierra de Huelva-Andévalo Central,479.0,180.0,235.0,67571.0,266.386468,347.782333,43.0
Lepe,230.0,53.0,92.0,27431.0,193.212059,335.386971,19.0
Almonte,125.0,36.0,61.0,24191.0,148.815675,252.159894,12.0
Cortegana,100.0,70.0,87.0,4666.0,1500.214316,1864.552079,9.0
Cartaya,72.0,30.0,41.0,19974.0,150.195254,205.266847,9.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granado (El),31.0,9.0,28.0,516.0,1744.186047,5426.356589,0.0
Cortegana,100.0,70.0,87.0,4666.0,1500.214316,1864.552079,9.0
Almonaster la Real,17.0,14.0,16.0,1819.0,769.653656,879.604178,5.0
Cerro de Andévalo (El),24.0,7.0,19.0,2364.0,296.108291,803.722504,2.0
Linares de la Sierra,2.0,2.0,2.0,263.0,760.456274,760.456274,1.0
Puerto Moral,2.0,2.0,2.0,282.0,709.219858,709.219858,0.0
Villarrasa,36.0,7.0,13.0,2176.0,321.691176,597.426471,2.0
Palma del Condado (La),142.0,29.0,50.0,10761.0,269.491683,464.640833,7.0
Valdelarco,3.0,1.0,1.0,239.0,418.410042,418.410042,0.0
